# CORD-19 Software Counting
This jupyter notebook is designated to count software mentions based on the CORD19 dataset from: 
https://datadryad.org/stash/dataset/doi:10.5061/dryad.vmcvdncs0

First, relevant packages must be imported to the Notebook. 

In [1]:
import numpy as np
import pandas as pd
import csv
import ast
import collections
import matplotlib.pyplot as plt
import Levenshtein as lev
from fuzzywuzzy import fuzz 

Get the data and save it to a variable. 

In [2]:
CORD19_CSV = pd.read_csv('../data/cord-19/CORD19_software_mentions.csv' , converters={'software': lambda x: x[1:-1].split(',')})

Show the head of the dataset to inspect all columns and obtain a broad overview. 

In [3]:
CORD19_CSV.head(20)

,paper_id,doi,title,source_x,license,publish_time,journal,url,software
0,00006903b396d50cc0037fed39916d57d50ee801,NaN,Urban green space and happiness in developed c...,ArXiv,arxiv,2021-01-04,NaN,https://arxiv.org/pdf/2101.00807v1.pdf,['Google Street View']
1,0000fcce604204b1b9d876dc073eb529eb5ce305,10.1016/j.regg.2021.01.002,La Geriatría de Enlace con residencias en la é...,Elsevier; PMC,els-covid,2021-01-13,Rev Esp Geriatr Gerontol,https://api.elsevier.com/content/article/pii/S...,['SEGG']
2,000122a9a774ec76fa35ec0c0f6734e7e8d0c541,10.1016/j.rec.2020.08.002,Impact of COVID-19 on ST-segment elevation myo...,Elsevier; Medline; PMC,no-cc,2020-09-08,Rev Esp Cardiol (Engl Ed),https://api.elsevier.com/content/article/pii/S...,"['STATA', 'IAMCEST']"
3,0001418189999fea7f7cbe3e82703d71c85a6fe5,10.1016/j.vetmic.2006.11.026,Absence of surface expression of feline infect...,Elsevier; Medline; PMC,no-cc,2007-03-31,Vet Microbiol,https://www.sciencedirect.com/science/article/...,['SPSS']
4,00033d5a12240a8684cfe943954132b43434cf48,10.3390/v12080849,Detection of Severe Acute Respiratory Syndrome...,Medline; PMC,cc-by,2020-08-04,Viruses,https://www.ncbi.nlm.nih.gov/pubmed/32759673/;...,"['R', 'MassARRAY Typer Analyzer']"
5,00035ac98d8bc38fbca02a1cc957f55141af67c0,10.3389/fpsyt.2020.559701,The Psychological Pressures of Breast Cancer P...,Medline; PMC,cc-by,2020-12-15,Front Psychiatry,https://doi.org/10.3389/fpsyt.2020.559701; htt...,"['Wechat', 'SPSS Statistics']"
6,00039b94e6cb7609ecbddee1755314bcfeb77faa,10.1111/j.1365-2249.2004.02415.x,Plasma inflammatory cytokines and chemokines i...,Medline; PMC,bronze-oa,2004-04-01,Clinical & Experimental Immunology,https://onlinelibrary.wiley.com/doi/pdfdirect/...,['Statistical Package for Social Sciences (SPS...
7,0004456994f6c1d5db7327990386d33c01cff32a,10.1186/1471-2334-10-8,Seasonal influenza risk in hospital healthcare...,PMC,cc-by,2010-01-12,BMC Infect Dis,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC2...,"['STATA', 'STATA', 'Statacorp']"
8,00073cb65dd2596249230fab8b15a71c4a135895,10.1086/605034,Risk Parameters of Fulminant Acute Respiratory...,Medline; PMC,no-cc,2009-08-01,J Infect Dis,https://doi.org/10.1086/605034; https://www.nc...,"['SPSS', 'SPSS']"
9,0007f972812bb45abbe5b0edf8db5359d49c23eb,10.1186/s42234-020-00057-1,The role of nicotinic receptors in SARS-CoV-2 ...,Medline; PMC,cc-by,2020-10-28,Bioelectron Med,https://www.ncbi.nlm.nih.gov/pubmed/33292872/;...,"['geNorm', 'GraphPad Prism', 'GraphPad', 'C..."


The dataset contains nine different columns.
Thusly, the next lines of this notebook explores the column "software". 
Therefore, the column software will be saved to a object. 

In [4]:
software = CORD19_CSV.software

In [5]:
software

0                    ['Google Street View']
1                                  ['SEGG']
2                     ['STATA',  'IAMCEST']
3                                  ['SPSS']
4        ['R',  'MassARRAY Typer Analyzer']
                        ...                
77443                          ['UpToDate']
77444                   ['SALib',  'Panda']
77445                ['Prism',  'GraphPad']
77446    ['R package circular',  'R',  'R']
77447       ['GRAM',  'R studio',  'Stata']
Name: software, Length: 77448, dtype: object

In [6]:
len_software_multiple_entries = len(software)
len_software_multiple_entries

77448

The software object contains 77448 rows. In each row, there are software entries. Some rows contain more than one software entry. For instance, row two and four have each two entries. As a result, the object needs to be transformed to an object which contains soley one software entry per row. 

In [7]:
software = software.explode(ignore_index = True)

Control the software object and inspect if each row contains only one entry. 

In [8]:
software

0         'Google Street View'
1                       'SEGG'
2                      'STATA'
3                    'IAMCEST'
4                       'SPSS'
                  ...         
558787                     'R'
558788                     'R'
558789                  'GRAM'
558790              'R studio'
558791                 'Stata'
Name: software, Length: 558792, dtype: object

In [9]:
len_software_single_entries = len(software)
len_software_single_entries

558792

Now, the object contains solely one entry per row and has the length of 558792.

In [10]:
average_entries_per_row = len_software_single_entries/len_software_multiple_entries
average_entries_per_row

7.215060427641773

Due to the alignment of the software object, it can be obtained that the dataset contains on average 7.2 software entries per row. Furthermore, the value_counts function will be user to minimise the amount of rows by checking identical duplicates. 

In [11]:
software.value_counts(dropna=False)

 'R'                                                     8389
 'SPSS'                                                  4736
'SPSS'                                                   4472
 'BLAST'                                                 3166
 'Excel'                                                 2666
                                                         ... 
 'VnCoreNLP'                                                1
 'ARCSINUS'                                                 1
 'Leica Application Suite Advanced Fluorescence Lite'       1
'Xenofontos'                                                1
'OLYMPUS MicroSuite'                                        1
Name: software, Length: 120279, dtype: int64

The function value_counts reduced the amount of rows to 120279. Nevertheless, the dtype counts same software mentions as distinct. For instance, "SPSS" is listed twice with two varied numbers. For this, the datatype will be converted to a dictionary to check for possible empty spaces.

In [12]:
software_dict = software.to_dict()
software_dict

{0: "'Google Street View'",
 1: "'SEGG'",
 2: "'STATA'",
 3: " 'IAMCEST'",
 4: "'SPSS'",
 5: "'R'",
 6: " 'MassARRAY Typer Analyzer'",
 7: "'Wechat'",
 8: " 'SPSS Statistics'",
 9: "'Statistical Package for Social Sciences (SPSS)'",
 10: " 'BD CBA'",
 11: "'STATA'",
 12: " 'STATA'",
 13: " 'Statacorp'",
 14: "'SPSS'",
 15: " 'SPSS'",
 16: "'geNorm'",
 17: " 'GraphPad Prism'",
 18: " 'GraphPad'",
 19: " 'Cellranger'",
 20: " 'R'",
 21: " 'Seurat'",
 22: " 'ggplot2'",
 23: " 'LinRegPCR'",
 24: "'GramA'",
 25: "'R package edgeR'",
 26: " 'R package edgeR'",
 27: " 'R package edgeR'",
 28: " 'STAR'",
 29: " 'FastQC'",
 30: " 'R package ALDEx2'",
 31: " 'ImageJ'",
 32: " 'PfAlbas'",
 33: " 'SAM'",
 34: "'MORO Praxis'",
 35: " 'MORO'",
 36: "'R2HC'",
 37: "'Singapour'",
 38: "'PRESET'",
 39: "'Google Trends (GT'",
 40: " 'GT'",
 41: "'SINUS'",
 42: " 'VICTIMES'",
 43: "'Spirocall'",
 44: " 'LIBSVM'",
 45: " 'MATLAB voicebox'",
 46: " 'openS'",
 47: " 'MILE'",
 48: "'DP'",
 49: " 'DP'",
 50: 

In this case, there are empty cases which make the function value_counts sum up software mentons with an additional empty space prior the string as distinct. Therefore, the function remove_empty_spaces(d) takes a dictionary and removes an empty space at the first position of a string. 

In [13]:
def remove_empty_spaces(dic):
    """ Function removing an empty space at the first position of a string. 
    """
    for i in dic:
        if dic[i][:1] == " ":
            dic[i] = dic[i][1:] #.strip() -> Improvement
    return dic

In [14]:
software_dict = remove_empty_spaces(software_dict)
software_dict

{0: "'Google Street View'",
 1: "'SEGG'",
 2: "'STATA'",
 3: "'IAMCEST'",
 4: "'SPSS'",
 5: "'R'",
 6: "'MassARRAY Typer Analyzer'",
 7: "'Wechat'",
 8: "'SPSS Statistics'",
 9: "'Statistical Package for Social Sciences (SPSS)'",
 10: "'BD CBA'",
 11: "'STATA'",
 12: "'STATA'",
 13: "'Statacorp'",
 14: "'SPSS'",
 15: "'SPSS'",
 16: "'geNorm'",
 17: "'GraphPad Prism'",
 18: "'GraphPad'",
 19: "'Cellranger'",
 20: "'R'",
 21: "'Seurat'",
 22: "'ggplot2'",
 23: "'LinRegPCR'",
 24: "'GramA'",
 25: "'R package edgeR'",
 26: "'R package edgeR'",
 27: "'R package edgeR'",
 28: "'STAR'",
 29: "'FastQC'",
 30: "'R package ALDEx2'",
 31: "'ImageJ'",
 32: "'PfAlbas'",
 33: "'SAM'",
 34: "'MORO Praxis'",
 35: "'MORO'",
 36: "'R2HC'",
 37: "'Singapour'",
 38: "'PRESET'",
 39: "'Google Trends (GT'",
 40: "'GT'",
 41: "'SINUS'",
 42: "'VICTIMES'",
 43: "'Spirocall'",
 44: "'LIBSVM'",
 45: "'MATLAB voicebox'",
 46: "'openS'",
 47: "'MILE'",
 48: "'DP'",
 49: "'DP'",
 50: "'DSGVO'",
 51: "'iOS'",
 52: 

Now, the software mentions within the dictionary do not contain empty spaces at the first position of the string. For the use of value_counts, the dictionary is converted to a pandas series. 

In [15]:
software_series = pd.Series(software_dict)
software_series.value_counts()

'R'                     10805
'SPSS'                   9208
'GraphPad Prism'         3986
'Excel'                  3856
'BLAST'                  3674
                        ...  
'ABI Prism'                 1
'PLUTO'                     1
'TransferWise'              1
'emplotment'                1
'OLYMPUS MicroSuite'        1
Length: 102740, dtype: int64

Due to the removing of empty spaces, the length of the dtype decreased. For the purpose of minimising the length of the dtype, all strings will be capitalized. 

In [16]:
software_dict = software_series.to_dict()

In [17]:
def capitalize_mentions(dic):
    """ Function iterating a dictionary and capitalizing all strings.
    """
    for i in dic:
        dic[i] = dic[i].upper()
    return dic

In [18]:
software_dict = capitalize_mentions(software_dict)
software_dict

{0: "'GOOGLE STREET VIEW'",
 1: "'SEGG'",
 2: "'STATA'",
 3: "'IAMCEST'",
 4: "'SPSS'",
 5: "'R'",
 6: "'MASSARRAY TYPER ANALYZER'",
 7: "'WECHAT'",
 8: "'SPSS STATISTICS'",
 9: "'STATISTICAL PACKAGE FOR SOCIAL SCIENCES (SPSS)'",
 10: "'BD CBA'",
 11: "'STATA'",
 12: "'STATA'",
 13: "'STATACORP'",
 14: "'SPSS'",
 15: "'SPSS'",
 16: "'GENORM'",
 17: "'GRAPHPAD PRISM'",
 18: "'GRAPHPAD'",
 19: "'CELLRANGER'",
 20: "'R'",
 21: "'SEURAT'",
 22: "'GGPLOT2'",
 23: "'LINREGPCR'",
 24: "'GRAMA'",
 25: "'R PACKAGE EDGER'",
 26: "'R PACKAGE EDGER'",
 27: "'R PACKAGE EDGER'",
 28: "'STAR'",
 29: "'FASTQC'",
 30: "'R PACKAGE ALDEX2'",
 31: "'IMAGEJ'",
 32: "'PFALBAS'",
 33: "'SAM'",
 34: "'MORO PRAXIS'",
 35: "'MORO'",
 36: "'R2HC'",
 37: "'SINGAPOUR'",
 38: "'PRESET'",
 39: "'GOOGLE TRENDS (GT'",
 40: "'GT'",
 41: "'SINUS'",
 42: "'VICTIMES'",
 43: "'SPIROCALL'",
 44: "'LIBSVM'",
 45: "'MATLAB VOICEBOX'",
 46: "'OPENS'",
 47: "'MILE'",
 48: "'DP'",
 49: "'DP'",
 50: "'DSGVO'",
 51: "'IOS'",
 52: 

In [19]:
software_series = pd.Series(software_dict)
software_series = software_series.value_counts()
software_series

'R'                                                   10805
'SPSS'                                                 9227
'GRAPHPAD PRISM'                                       4461
'EXCEL'                                                4054
'BLAST'                                                3943
                                                      ...  
'PIAR'                                                    1
'OPEN STREET MAP TRANSIT FEED SPECIFICATION (GTFS'        1
'ELOQUENT'                                                1
'THUNDERSTORM DESCRIPTION LANGUAGE TDL'                   1
'CATTAILS'                                                1
Length: 89498, dtype: int64

Due to the capitalization of software mentions, the length of the dytpe could be decreased. Subsequently, the fuzzywuzzy compare algorithm will be introduced. This algorithm is based on Levenshtein which checks the similarity of strings by various aspects. 

In [20]:
def fuzzy_ratio_compare(str1, str2, th):
    """ Function to compare to strings based on a given threeshold. 
    """
    ratio = fuzz.ratio(str1, str2)
    if(ratio > th):
        return True
    else:
        return False

Due for performance reasons, further investigation will be conducted with a subset of the initial series based on the limit. 

In [21]:
limit = 1000
software_series_shaped = software_series.head(limit)
software_series_shaped

'R'                 10805
'SPSS'               9227
'GRAPHPAD PRISM'     4461
'EXCEL'              4054
'BLAST'              3943
                    ...  
'CUSTOMMUNE'           69
'MEDISYS'              69
'DPLYR'                69
'FACEBOOK LIVE'        69
'GRAPHPPI'             69
Length: 1000, dtype: int64

Converting the series to a DataFrame for comparison purposes. The index of the DataFrame is required for selecting rows. 

In [22]:
ts = software_series_shaped.to_frame()
list_soft = []
list_matches = [0]
for i in range(len(ts)):
    list_soft.append(software_series_shaped.index[i])
    list_matches.append(software_series_shaped[i])
df_shaped = pd.DataFrame()
df_shaped['Software'] = list_soft
df_shaped['Matches'] = list_matches[1:]
df_shaped

,Software,Matches
0,'R',10805
1,'SPSS',9227
2,'GRAPHPAD PRISM',4461
3,'EXCEL',4054
4,'BLAST',3943
...,...,...
995,'CUSTOMMUNE',69
996,'MEDISYS',69
997,'DPLYR',69
998,'FACEBOOK LIVE',69


In [23]:
# def show_matches(str1, str2, int1, int2):
#     """Show matched strings linked to ratio
#     """
#     int1 = str(int1)
#     int2 = str(int2)   
#     show_matches(dfa['Software'][i], str(dfa['Matches'][i]), dfb['Software'][j], str(dfb['Matches'][j])
#     print(str1 + " with " + str(int1) + "entries matched with " + str2 + " with " + str(int2) + " entries") 

Replacing special characters to prevent unterminated subpattern at a later stage of this Notebook.

In [24]:
df_shaped['Software'] = df_shaped.Software.str.replace('(','')
df_shaped['Software'] = df_shaped.Software.str.replace(')','')

The following function compares software mentions based on the fuzzywuzzy method. As a result, a blacklist with identified duplicates and a modified dataframe are returned. 

In [25]:
def unify_dataframe(df, th):
    """Match software mentions based fuzzywuzzy algorithm
    """
    dfa = df
    dfb = df
    df_holder = df
    blacklist = set()
    for i in range(len(dfa)):
        for j in range(i + 1, len(dfb)):
            if dfb['Software'][j] not in blacklist:
                if(fuzzy_ratio_compare(dfa['Software'][i], dfb['Software'][j], th)):
                    print(dfa['Software'][i]+" " + str(dfa['Matches'][i]) + " matched " + dfb['Software'][j] + " " + str(dfb['Matches'][j]))
                    dfa['Matches'][i] = int(dfa['Matches'][i]+ dfb['Matches'][j])
                    blacklist.add(dfb['Software'][j])
                    df_holder = dfa
    return df_holder, blacklist

In [26]:
%%time
threeshold = 90
df_blacklist = unify_dataframe(df_shaped, threeshold)
df_unified = df_blacklist[0]
blacklist = df_blacklist[1]
#Wall time: 1h 4min 32s -> head = 10000 th -> 85
#Wall time: 17min 25s -> head = 5000 th -> 85

'GRAPHPAD PRISM' 4461 matched 'GRAPH PAD PRISM' 156
'BLAST' 3943 matched 'BLASTN' 1075
'BLAST' 5018 matched 'BLASTP' 665
'BLAST' 5683 matched 'BLASTX' 617


<ipython-input-25-d8561234f01a>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfa['Matches'][i] = int(dfa['Matches'][i]+ dfb['Matches'][j])


'BLAST' 6300 matched 'BLAT' 76
'STATA' 3688 matched 'STAT' 210
'MEGA' 2118 matched 'MEGA6' 205
'MEGA' 2323 matched 'MEGA7' 205
'MEGA' 2528 matched 'MEGAN' 152
'MEGA' 2680 matched 'MEGA5' 137
'MEGA' 2817 matched 'MEGA4' 96
'MEGA' 2913 matched 'OMEGA' 73
'NET' 2041 matched 'UNET' 164
'IMAGEJ' 1949 matched 'IMAGE J' 509
'IMAGEJ' 2458 matched 'IMAGE' 236
'PRISM' 1440 matched 'PRISMA' 566
'CLUSTALW' 1104 matched 'CLUSTAL W' 385
'CLUSTALW' 1489 matched 'CLUSTAL' 231
'CLUSTALW' 1720 matched 'CLUSTALW2' 78
'BLASTN' 1075 matched 'TBLASTN' 141
'BEAST' 940 matched 'BEST' 117
'AUTODOCK' 920 matched 'AUTODOCK4' 86
'ENSEMBL' 800 matched 'ENSEMBLE' 167
'BERT' 717 matched 'MBERT' 104
'BLASTX' 617 matched 'TBLASTX' 173
'CHIMERA' 501 matched 'CHIMAERA' 128
'DESEQ2' 492 matched 'DESEQ' 82
'BOWTIE2' 489 matched 'BOWTIE' 229
'GOOGLE FORMS' 472 matched 'GOOGLE FORM' 88
'MOE' 429 matched 'MOEA' 124
'RSTUDIO' 419 matched 'R STUDIO' 148
'RSTUDIO' 567 matched 'STUDIO' 106
'ONE' 416 matched 'CONE' 103
'MFOLD' 38

The blacklist contains the matched duplicates which means that they need to be removed from the dataframe.

In [27]:
for i in blacklist: 
    df_unified = df_unified[~df_unified['Software'].str.contains(i)]

For comparison purposes, the DataFrame is sorted in descending order by matches. 

In [28]:
df_unified.sort_values(by=['Matches'], inplace=True, ascending=False)
df_unified

,Software,Matches
0,'R',10805
1,'SPSS',9227
4,'BLAST',6376
2,'GRAPHPAD PRISM',4617
3,'EXCEL',4054
...,...,...
995,'CUSTOMMUNE',69
996,'MEDISYS',69
997,'DPLYR',69
998,'FACEBOOK LIVE',69


To verify the removal of duplicates, the length of the dataframe is outputed.  

In [29]:
len(df_unified)

896

To investigate the postion change of software mentions, the following algorithm compares its index postion to the sorted postion by matches.

In [34]:
list_change = []
for i in range(len(df_unified)):
    dif = df_unified.index[i]-i
    if(dif > 0):
        list_change.append("+"+str(dif))
    else:
        list_change.append(df_unified.index[i]-i)
df_unified['Change'] = list_change
df_unified.head(50)

,Software,Matches,Change
0,'R',10805,0
1,'SPSS',9227,0
4,'BLAST',6376,+2
2,'GRAPHPAD PRISM',4617,-1
3,'EXCEL',4054,-1
5,'STATA',3898,0
6,'SAS',3151,0
10,'MEGA',2986,+3
12,'IMAGEJ',2694,+4
7,'MATLAB',2653,-2
